In [8]:
%load_ext autoreload
%autoreload 2\

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [9]:
# Libraries
import os
import pickle

from src.models import train_model, test_model

In [10]:
# os.chdir('/Users/magdalena/OneDrive - Queen Mary, University of London/bezzlab/research/projects/phospho_pi/')
os.chdir('/home/mhuebner/Desktop/bezzlab/research/projects/phospho_pi/')

In [11]:
EXPERIMENT = 'mh05'
MODEL = 'model_A1'
EXP_DIR = f'./models/{EXPERIMENT}/'
# Training
PATH_TO_DATA = f'data/processed/synthetic_data/{MODEL}/'
# Models
model_template = f'./models/templates/{MODEL}.pl'

In [12]:
experiments = [25, 50, 100, 250, 500, 1000]  # 25 samples not enough when using testing data, can't predict occupancy, therefore equal probs, no mpe, duplicates removed --> empty facts list, can't calculate anything

In [ ]:
for n_samples in experiments:
    path_to_data = f'{PATH_TO_DATA}n{n_samples}/mutate_m1_20/'
    train_model.main(
        experiment = EXPERIMENT, 
        path_to_data = path_to_data, 
        model_template = model_template, 
        training = 0.8, 
        max_iter = 3, 
        evidence = ['activity', 'occupancy', 'fold_change'],
        facts = [],
        suffix = f'_n{n_samples}'
    )

In [13]:
# Testing
test = ['training', 'testing']
results = {}

In [ ]:
for n_samples in experiments:
    PATH_TO_TRUTH = f'data/processed/synthetic_data/{MODEL}/n{n_samples}/'
    model_trained = f'{EXP_DIR}model_trained_n{n_samples}.pl'
    results[f'n{n_samples}'] = {}
    for test_on in test:
        PATH_TO_TESTING = f'{EXP_DIR}{test_on}_n{n_samples}/'
        compare_to = [test_on, 'truth']
        for compare in compare_to:
            if compare == 'truth':
                path_to_truth = PATH_TO_TRUTH
            else: 
                path_to_truth = PATH_TO_TESTING

            test_results = test_model.main(
                experiment = EXPERIMENT, 
                path_to_data = PATH_TO_TESTING, 
                path_to_truth = path_to_truth, 
                model_trained = model_trained, 
                targets = ['occupancy', 'modification'], 
                test_on = f'{test_on}_n{n_samples}', compare = f'{compare}_n{n_samples}')
            results[f'n{n_samples}'][f'{test_on}_vs_{compare}'] = test_results

In [15]:
with open(f'{EXP_DIR}results.pickle', 'wb') as file:
    pickle.dump(results, file, protocol=pickle.HIGHEST_PROTOCOL)